# Fix Poor Predictions from Comprehend Custom Text Classifier

In [11]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [12]:
import io
import json
import uuid
import time
import boto3
import botocore

# Amazon Python SDK clients
sagemaker = boto3.client('sagemaker', region)
comprehend = boto3.client('comprehend', region)
a2i = boto3.client('sagemaker-a2i-runtime')
s3 = boto3.client('s3', region)

# Retrieve the `augmented_ai_flow_definition_arn` Created Previously

In [13]:
%store -r augmented_ai_flow_definition_arn

In [14]:
print(augmented_ai_flow_definition_arn)

arn:aws:sagemaker:us-west-2:032934710550:flow-definition/fd-dsoaws-comprehend-d396f583-deed-43f0-a198-286fffb30c3e


# _Wait for the Comprehend Job to Complete from the Previous Section_

# Retrieve the `comprehend_endpoint_arn` Deployed Previously

In [15]:
%store -r comprehend_endpoint_arn

In [16]:
try: 
    comprehend_endpoint_arn
except NameError:
    print('*** PLEASE WAIT FOR THE Comprehend JOB TO FINISH IN THE PREVIOUS SECTION BEFORE CONTINUING ***')
    print('*** YOU WILL NEED TO RESTART THIS NOTEBOOK ONCE THE JOB FINISHES ***')

In [17]:
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-west-2:032934710550:document-classifier-endpoint/comprehend-inference-ep-22-18-13-36


# Check the Confidence Score for Each Comprehend Prediction
If < threshold, start the human loop.  You can integrate this type of logic into your application using the SDK.  In this case, we're using the Python SDK.

# Use Comprehend to Predict Some Sample Reviews

In [18]:
sample_reviews = [
                  'I enjoy this product', 
                  'I am unhappy with this product', 
                  'It is okay', 
                  'sometimes it works'
                 ]

# Start a Human Loop When Comprehend Does Not Produce a Confident Prediction
The human lmoop will engage our workforce for human review if the confidence of the Comprehend prediction is less than the provided confidence.

![](img/augmented-ai-comprehend-predictions.png)

In [19]:
human_loops_started = []

CONFIDENCE_SCORE_THRESHOLD = 0.90

for sample_review in sample_reviews:
    # Call the Comprehend Custom model that we trained earlier
    response = comprehend.classify_document(Text=sample_review, 
                                            EndpointArn=comprehend_endpoint_arn)

    star_rating = response['Classes'][0]['Name']
    confidence_score = response['Classes'][0]['Score']
    
    print(f'Processing sample_review: \"{sample_review}\"')
    
    # Our condition for when we want to engage a human for review
    if (confidence_score < CONFIDENCE_SCORE_THRESHOLD):
    
        humanLoopName = str(uuid.uuid4())
        inputContent = {
            'initialValue': star_rating,
            'taskObject': sample_review
        }
        start_loop_response = a2i.start_human_loop(
            HumanLoopName=humanLoopName,
            FlowDefinitionArn=augmented_ai_flow_definition_arn,
            HumanLoopInput={
                'InputContent': json.dumps(inputContent)
            }
        )

        human_loops_started.append(humanLoopName)

        print(f'Confidence score of {confidence_score} for star rating of {star_rating} is less than the threshold of {CONFIDENCE_SCORE_THRESHOLD}')
        print(f'*** ==> Starting human loop with name: {humanLoopName}  \n')
    else:
        print(f'Confidence score of {confidence_score} for star rating of {star_rating} is above threshold of {CONFIDENCE_SCORE_THRESHOLD}')
        print('No human loop created. \n')

Processing sample_review: "I enjoy this product"
Confidence score of 0.3887999951839447 for star rating of 5 is less than the threshold of 0.9
*** ==> Starting human loop with name: 21bc7985-446d-4e12-8f09-1fa273b3a214  

Processing sample_review: "I am unhappy with this product"
Confidence score of 0.4156000018119812 for star rating of 1 is less than the threshold of 0.9
*** ==> Starting human loop with name: 8faac08a-4bd5-4e1f-af59-faf125292468  

Processing sample_review: "It is okay"
Confidence score of 0.4462999999523163 for star rating of 3 is less than the threshold of 0.9
*** ==> Starting human loop with name: 3210fe4c-139b-45b9-bc4c-ee5e8e14919f  

Processing sample_review: "sometimes it works"
Confidence score of 0.42089998722076416 for star rating of 4 is less than the threshold of 0.9
*** ==> Starting human loop with name: 06beb809-6e92-4b5a-bb16-38ad7cd34593  



# Check Status of Human Loop

In [20]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('')
        
    if resp['HumanLoopStatus'] == 'Completed':
        completed_human_loops.append(resp)

HumanLoop Name: 21bc7985-446d-4e12-8f09-1fa273b3a214
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-west-2-032934710550/a2i-comprehend-star-rating-results/fd-dsoaws-comprehend-d396f583-deed-43f0-a198-286fffb30c3e/2020/08/22/18/22/30/21bc7985-446d-4e12-8f09-1fa273b3a214/output.json'}

HumanLoop Name: 8faac08a-4bd5-4e1f-af59-faf125292468
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-west-2-032934710550/a2i-comprehend-star-rating-results/fd-dsoaws-comprehend-d396f583-deed-43f0-a198-286fffb30c3e/2020/08/22/18/22/30/8faac08a-4bd5-4e1f-af59-faf125292468/output.json'}

HumanLoop Name: 3210fe4c-139b-45b9-bc4c-ee5e8e14919f
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-west-2-032934710550/a2i-comprehend-star-rating-results/fd-dsoaws-comprehend-d396f583-deed-43f0-a198-286fffb30c3e/2020/08/22/18/22/30/3210fe4c-139b-45b9-bc4c-ee5e8e14919f/output.json'}

Huma

# Wait For Workers to Complete Their Human Loop Tasks

Navigate to the link below and login with your email and password that you used when you set up the Private Workforce.

In [21]:
%store -r augmented_ai_workteam_arn

In [22]:
print(augmented_ai_workteam_arn)

arn:aws:sagemaker:us-west-2:032934710550:workteam/private-crowd/dsoaws


In [23]:
workteam_name = augmented_ai_workteam_arn[augmented_ai_workteam_arn.rfind('/') + 1:]
print(workteam_name)
print('Navigate to the private worker portal and complete the human loop.')
print('Make sure you have invited yourself to the workteam and received the signup email.')
print('Note:  Check your spam filter if you have not received the email.')
print('')
print('https://' + sagemaker.describe_workteam(WorkteamName=workteam_name)['Workteam']['SubDomain'])

dsoaws
Navigate to the private worker portal and complete the human loop.
Make sure you have invited yourself to the workteam and received the signup email.
Note:  Check your spam filter if you have not received the email.

https://65u72v6gtb.labeling.us-west-2.sagemaker.aws


# Start Labeling

<img src="img/augmented-comprehend-custom-start-working.png" width="80%" align="left">

# Select Label

<img src="img/augmented-comprehend-custom-select-label.png" width="80%" align="left">

# Loop is Completed

<img src="img/augmented-comprehend-custom-finished-task.png" width="80%" align="left">

# Verify the Human Loops are Completed

In [24]:
import time

completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('')
    while resp["HumanLoopStatus"] != "Completed":
        print(f'Waiting for HumanLoop to complete.')  
        time.sleep(10)
        resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)
        print(f'Completed!')
        print('')

HumanLoop Name: 21bc7985-446d-4e12-8f09-1fa273b3a214
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-west-2-032934710550/a2i-comprehend-star-rating-results/fd-dsoaws-comprehend-d396f583-deed-43f0-a198-286fffb30c3e/2020/08/22/18/22/30/21bc7985-446d-4e12-8f09-1fa273b3a214/output.json'}

Waiting for HumanLoop to complete.
Waiting for HumanLoop to complete.
Waiting for HumanLoop to complete.
Waiting for HumanLoop to complete.
Completed!

HumanLoop Name: 8faac08a-4bd5-4e1f-af59-faf125292468
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-west-2-032934710550/a2i-comprehend-star-rating-results/fd-dsoaws-comprehend-d396f583-deed-43f0-a198-286fffb30c3e/2020/08/22/18/22/30/8faac08a-4bd5-4e1f-af59-faf125292468/output.json'}

Waiting for HumanLoop to complete.
Completed!

HumanLoop Name: 3210fe4c-139b-45b9-bc4c-ee5e8e14919f
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sage

# View Human Labels  

Once the work is complete, Amazon A2I stores the results in the specified S3 bucket and sends a Cloudwatch Event.  Let's check the S3 contents.

In [25]:
import re
import pprint

pp = pprint.PrettyPrinter(indent=4)

fixed_items = []

for resp in completed_human_loops:
    split_string = re.split('s3://' +  bucket + '/', resp['HumanLoopOutput']['OutputS3Uri'])
    output_bucket_key = split_string[1]

    response = s3.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response['Body'].read().decode('utf-8')
    json_output = json.loads(content)
    print(json_output)

    input_content = json_output['inputContent']
    human_answer = json_output['humanAnswers'][0]['answerContent']
    fixed_item = {'input_content': input_content, 'human_answer': human_answer}
    fixed_items.append(fixed_item)

{'flowDefinitionArn': 'arn:aws:sagemaker:us-west-2:032934710550:flow-definition/fd-dsoaws-comprehend-d396f583-deed-43f0-a198-286fffb30c3e', 'humanAnswers': [{'answerContent': {'star_rating': {'label': '5'}}, 'submissionTime': '2020-08-22T18:23:03.664Z', 'workerId': '2fe1fbc789375c43', 'workerMetadata': {'identityData': {'identityProviderType': 'Cognito', 'issuer': 'https://cognito-idp.us-west-2.amazonaws.com/us-west-2_Un0GovN9y', 'sub': 'c26c3601-857d-40ea-bc05-a7f3a432f72c'}}}], 'humanLoopName': '21bc7985-446d-4e12-8f09-1fa273b3a214', 'inputContent': {'initialValue': '5', 'taskObject': 'I enjoy this product'}}
{'flowDefinitionArn': 'arn:aws:sagemaker:us-west-2:032934710550:flow-definition/fd-dsoaws-comprehend-d396f583-deed-43f0-a198-286fffb30c3e', 'humanAnswers': [{'answerContent': {'star_rating': {'label': '1'}}, 'submissionTime': '2020-08-22T18:23:12.285Z', 'workerId': '2fe1fbc789375c43', 'workerMetadata': {'identityData': {'identityProviderType': 'Cognito', 'issuer': 'https://cogni

# Prepare the Data for Re-training

In [26]:
df_fixed_items = pd.DataFrame(fixed_items)

In [27]:
df_fixed_items.head()

,input_content,human_answer
0,"{'initialValue': '5', 'taskObject': 'I enjoy t...",{'star_rating': {'label': '5'}}
1,"{'initialValue': '1', 'taskObject': 'I am unha...",{'star_rating': {'label': '1'}}
2,"{'initialValue': '3', 'taskObject': 'It is okay'}",{'star_rating': {'label': '3'}}
3,"{'initialValue': '4', 'taskObject': 'sometimes...",{'star_rating': {'label': '3'}}


# Once finished, delete the Comprehend Custom Model Endpoint

In [28]:
# comprehend.delete_endpoint(EndpointArn=comprehend_endpoint_arn)

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();